In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import os
import numpy as np
from operator import itemgetter 
from random import shuffle 
import optuna
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [ ]:
files = []
classes = ['kiss', 'handshake', 'hug', 'highfive']

for class_n, c in enumerate(classes):
    f = os.listdir('output/'+c)
    f = list(map(lambda x: 'output/'+c+'/'+x, f))
    
    filtered = list(filter(lambda x: x.split('.')[-1] == 'npy', f))
    
    files.extend(filtered)

In [ ]:
def convertClass(classname):
    if classname == None:
        raise Error('No classname!')
        
    mapping = {key: value for (value, key) in enumerate(classes)}
    
    return mapping[classname]

def onehot(y):
    onehot = np.zeros((4))
    onehot[y] = 1
    return tf.constant(onehot)

In [ ]:
test_size = 0.2

shuffled_idx = np.random.permutation(len(files))
split_at = int(len(shuffled_idx) * (1-test_size))

train_size = len(shuffled_idx[:split_at])
test_size = len(shuffled_idx[split_at:])

In [ ]:
epochs = 20

def batcherize(files, train=True):
    if train:
        indices = shuffled_idx[:split_at]
    else:
        indices = shuffled_idx[split_at:]
        
    for _ in range(epochs+1):
        for idx in indices:
            df = np.load(files[idx])/256
            class_name = files[idx].split('/')[-2]
            class_int = convertClass(class_name)
            class_onehot = onehot(class_int)

            yield np.array([df]), np.array([class_onehot])
            

In [ ]:
epochs = 20

def batcherize_train(files=files, train=True):
    if train:
        indices = shuffled_idx[:split_at]
    else:
        indices = shuffled_idx[split_at:]
        
    for _ in range(epochs+1):
        for idx in indices:
            df = np.load(files[idx])/256
            class_name = files[idx].split('/')[-2]
            class_int = convertClass(class_name)
            class_onehot = onehot(class_int)

            yield tf.constant(df), class_onehot
            
def batcherize_test(files=files, train=False):
    if train:
        indices = shuffled_idx[:split_at]
    else:
        indices = shuffled_idx[split_at:]
        
    for _ in range(epochs+1):
        for idx in indices:
            df = np.load(files[idx])/256
            class_name = files[idx].split('/')[-2]
            class_int = convertClass(class_name)
            class_onehot = onehot(class_int)

            yield tf.constant(df), class_onehot
            

In [ ]:
tf_train = tf.data.Dataset.from_generator(batcherize_train, output_types=(tf.dtypes.float32, tf.dtypes.int8))\
.padded_batch(4, padded_shapes=([None, 128, 128, 3], ([4])))

tf_test = tf.data.Dataset.from_generator(batcherize_test, output_types=(tf.dtypes.float32, tf.dtypes.int8))\
.padded_batch(4, padded_shapes=([None, 128, 128, 3], ([4])))

In [ ]:
model = Sequential()

model.add(InputLayer((None, 128, 128,3)))

model.add(TimeDistributed(
    Conv2D(
        filters=8, 
        kernel_size=6, 
        input_shape=(128, 128, 3), 
        data_format='channels_last', 
        padding="same",
    ), 
    name='Conv2D'))

model.add(TimeDistributed(MaxPooling2D(pool_size=2), name='MaxPooling2D'))
model.add(TimeDistributed(Flatten()))

model.add(GRU(32, dropout=0.3, recurrent_dropout=0.4, kernel_regularizer=tf.keras.regularizers.L2(0.005)))

model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.L2(0.1)))
model.add(Dropout(0.3))

model.add(Dense(4, activation='softmax'))

model.summary()

In [ ]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)
early = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, min_delta=0.02)

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])


h = model.fit(
    tf_train,
    epochs=epochs, 
    batch_size=4,
    steps_per_epoch=train_size/4,
    validation_data=tf_test,
    validation_steps=test_size/4,
    callbacks=[scheduler]
)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16,4))

ax[0].plot(h.history['loss'], label='Training')
ax[0].plot(h.history['val_loss'], label='Test')
ax[0].set_title('Loss')
ax[0].legend()

ax[1].plot(h.history['accuracy'], label='Training')
ax[1].plot(h.history['val_accuracy'], label='Test')
ax[1].set_title('Accuracy')
ax[1].legend()